In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [271]:
import pandas as pd
import numpy as np

In [502]:
df = pd.read_csv('/kaggle/input/traindata/Train.csv')
df = df.set_index('NO')


In [503]:
df.shape

(281, 78)

หา missing data

In [504]:
df.isnull().values.any()

True

In [505]:
check = df.isnull().sum(axis =0).values
check 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [506]:
df_clean_column = df.drop(['Q31'],axis = 1)
df_clean = df_clean_column.dropna(axis = 0)

In [507]:
check = df_clean.isnull().sum(axis =0).values
check 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [508]:
pel_class = np.unique(df_clean['CLASS'])
pel_class

array([1, 2, 3])

# ตั้ง Train & test

In [512]:
y = df_clean['CLASS'].values
X = df_clean.drop(['CLASS'],axis = 1).values

In [513]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X, y)


In [514]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=1)

In [515]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
fs = SelectKBest(chi2, k=65)
fs.fit(X, y)
X_train = fs.transform(X_train)
X_test = fs.transform(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

# SVM

In [488]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [1,2,4,8,16,32], # High C = Overfitting
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1] # High gamma = Overfitting
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

Best params : {'C': 4, 'gamma': 0.25}
10CV accuracy : 92.52840909090911


In [489]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
target_names = ['negative', 'positive']
sum(y_test == y_predict)/len(y_test)

0.8765432098765432

# Testing

In [496]:
df_t = pd.read_csv('/kaggle/input/testdata/Test.csv')
df_t = df_t.set_index('NO')
df_Tclean_column = df_t.drop(['Q31'],axis = 1)
df_test_Clean = df_Tclean_column.dropna(axis = 0)

In [497]:
df_test_Clean.shape

(30, 77)

In [498]:
y_test_data = df_test_Clean['CLASS'].values
X_test_data = df_test_Clean.drop(['CLASS'],axis = 1).values

In [499]:
X_test_data = fs.transform(X_test_data)



In [501]:
y_predict = clf.predict(X_test_norm_test)
print("Test accuracy : "+str(sum(y_test_data == y_predict)/len(y_test_data)*100))

Test accuracy : 60.0
